# How to extract information from parent GO terms

* **1) Load the GO DAG**
* **2) Pick a GO term and visualize**
  * **2a) Print GO information**
  * **2b) Plot GO term**
* **3. Find GO parents for numerous GO IDs**
  * **3a. Find GO parents up all relationships**
  * **3b. Find GO parents up *is_a* relationships only**
* **4. Find GO parents by traversing explicit relationships**

<a id='section1'></a>
## 1. Load the GO DAG
Required relationship between GO terms, *is_a*, is always loaded.

Load optional relationships, like *part_of*, so we have the option of finding GO parents with or without relationships.

In [1]:
from goatools.base import get_godag
godag = get_godag('go-basic.obo', optional_attrs='relationship')

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms; optional_attrs(relationship)


<a id='section2'></a>
## 2. Pick a GO term and visualize

In [2]:
goid = 'GO:0050807'

<a id='section2a'></a>
### 2a. Print GO information

In [3]:
def prt_flds(gosubdag):
    """Print the available printing fields"""
    print('Print fields:')
    for fld in sorted(gosubdag.prt_attr['flds']):
        print('    {F}'.format(F=fld))

In [4]:
from goatools.gosubdag.gosubdag import GoSubDag

# Create a subset of the GO DAG which contains:
#   * The selected GO term and
#   * All the GO terms above it
gosubdag = GoSubDag(goid, godag, relationships=True, prt=False)

# Get additional information for chosen GO
ntgo = gosubdag.go2nt[goid]

# Choose fields and custom printing format
# prt_flds(gosubdag)  # Uncomment to see the available print fields
prtfmt = '{NS} {GO} D{depth:02} {GO_name}'

# Print detailed information for GO
print(prtfmt.format(**ntgo._asdict()))

BP GO:0050807 D05 regulation of synapse organization


<a id='section2b'></a>
### 2b. Plot GO term

In [5]:
from goatools.gosubdag.plot.gosubdag_plot import GoSubDagPlot
GoSubDagPlot(gosubdag).plt_dag('reg_synapse_org.png')

    1 usr  12 GOs  WROTE: reg_synapse_org.png


![regulation of synapse organization](images/reg_synapse_org.png)

<a id='section3'></a>
## 3. Find GO parents for numerous GO IDs
### 3a. Find GO parents up all relationships

In [6]:
from goatools.godag.go_tasks import get_go2parents

go2parents = get_go2parents(gosubdag.go2obj, gosubdag.relationships)
for goid_parent in go2parents[goid]:
    print(prtfmt.format(**gosubdag.go2nt[goid_parent]._asdict()))

BP GO:0051128 D04 regulation of cellular component organization
BP GO:0050803 D03 regulation of synapse structure or activity
BP GO:0050808 D03 synapse organization


<a id='section3b'></a>
### 3b. Find GO parents up *is_a* relationships only

In [7]:
from goatools.godag.go_tasks import get_go2parents_isa

go2parents = get_go2parents_isa(gosubdag.go2obj)
for goid_parent in go2parents[goid]:
    print(prtfmt.format(**gosubdag.go2nt[goid_parent]._asdict()))

BP GO:0051128 D04 regulation of cellular component organization


<a id='section4'></a>
## 4. Find GO parents by traversing explicit relationships

In [8]:
goterm = godag[goid]

print('Parents up "is_a": required relationship')
for p_term in goterm.parents:
    print(prtfmt.format(**gosubdag.go2nt[p_term.item_id]._asdict()))
    
if 'part_of' in goterm.relationship:
    print('\nParents up "part_of" optional relationship:')
    for p_go in goterm.relationship['part_of']:
        print(prtfmt.format(**gosubdag.go2nt[p_go.item_id]._asdict()))
        
if 'regulates' in goterm.relationship:
    print('\nParents up "regulates" optional relationship:')
    for p_go in goterm.relationship['regulates']:
        print(prtfmt.format(**gosubdag.go2nt[p_go.item_id]._asdict()))

# godag must be loaded with: optional_attrs='relationship'
# gosubdag must be loaded with: relationships=True
print('\nAncestors up all loaded relationships:')
for p_go in gosubdag.rcntobj.go2ancestors[goid]:
    print(prtfmt.format(**gosubdag.go2nt[p_go]._asdict()))

Parents up "is_a": required relationship
BP GO:0051128 D04 regulation of cellular component organization

Parents up "part_of" optional relationship:
BP GO:0050803 D03 regulation of synapse structure or activity

Parents up "regulates" optional relationship:
BP GO:0050808 D03 synapse organization

Ancestors up all loaded relationships:
BP GO:0050789 D02 regulation of biological process
BP GO:0050794 D03 regulation of cellular process
BP GO:0050803 D03 regulation of synapse structure or activity
BP GO:0016043 D02 cellular component organization
BP GO:0065008 D02 regulation of biological quality
BP GO:0050808 D03 synapse organization
BP GO:0008150 D00 biological_process
BP GO:0009987 D01 cellular process
BP GO:0065007 D01 biological regulation
BP GO:0051128 D04 regulation of cellular component organization
BP GO:0071840 D01 cellular component organization or biogenesis


Copyright (C) 2019-present, DV Klopfenstein, H Tang. All rights reserved.